In [1]:
import matplotlib as mpl
mpl.use('pgf')

mpl.rcParams.update({
    'pgf.texsystem': 'pdflatex',
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,  
})

import matplotlib.pyplot as plt
# Show the image with ipywidges
import matplotlib.dates as mdates

import pandas as pd
import datetime
import json
import numpy as np

In [2]:
import tempfile
import zipfile
import os

with open('../batch_processing/processed_things_2024_01_22.zip', 'rb') as f:
    # Unzip in tempdir and load json
    with tempfile.TemporaryDirectory() as temp_dir:
        with zipfile.ZipFile(f, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        with open(os.path.join(temp_dir, "processed_things_2024_01_22.json")) as f:
            processed_things = json.load(f)
    
distances_filtered = []
noise_filtered = []

hours = 0
hours_vert = 0
hours_hor = 0
hours_with_minimal_instability = 0

total_cycles_count_zero = 0
total_things_count = 0
removed_traffic_lights_count = 0

buckets_with_greater_cycle_discrepancy = 0
buckets_with_greater_wait_time_diversity = 0
buckets_overall_hard_to_predict = 0

for thing_name, thing in processed_things.items():
    if thing["TotalCyclesCount"] == 0:
        total_cycles_count_zero += 1
        continue
    total_things_count += 1
    if thing["TotalRemovedCycleCount"] / thing["TotalCyclesCount"] > 0.1:
        removed_traffic_lights_count += 1
        continue
    for day_idx in range(7):
        for hour_idx in range(24):
            if thing["Metrics"][day_idx][hour_idx] != -1.0 and thing["ShiftsFuzzyness"][day_idx][hour_idx] != -1.0:
                hours += 1
                n = thing["ShiftsFuzzyness"][day_idx][hour_idx]
                d = thing["Metrics"][day_idx][hour_idx]
                if n < 0.1 and d < 5:
                    hours_with_minimal_instability += 1
                if d == 0 and n > 0.1:
                    hours_vert += 1
                if n == 1:
                    hours_hor += 1
                    
                if d > 5:
                    buckets_with_greater_cycle_discrepancy += 1
                if n > 0.2:
                    buckets_with_greater_wait_time_diversity += 1
                if d > 5 and n > 0.2:
                    buckets_overall_hard_to_predict += 1
                
                distances_filtered.append(thing["Metrics"][day_idx][hour_idx])
                noise_filtered.append(thing["ShiftsFuzzyness"][day_idx][hour_idx])
                
print("Total cycles count zero: ", total_cycles_count_zero)
print("Total things count: ", total_things_count)
print("Removed traffic lights count: ", removed_traffic_lights_count)

print("Hours with minimal instability: ", hours_with_minimal_instability)
print("Hours vertical: ", hours_vert)
print("Hours horizontal: ", hours_hor)
print("Hours: ", hours)

print("Buckets with greater cycle discrepancy: ", buckets_with_greater_cycle_discrepancy / hours)
print("Buckets with greater wait time diversity: ", buckets_with_greater_wait_time_diversity / hours)
print("Buckets overall hard to predict: ", buckets_overall_hard_to_predict / hours)

Total cycles count zero:  21160
Total things count:  18528
Removed traffic lights count:  2106
Hours with minimal instability:  1250302
Hours vertical:  224182
Hours horizontal:  72409
Hours:  2495574
Buckets with greater cycle discrepancy:  0.328952377288752
Buckets with greater wait time diversity:  0.2260201460665963
Buckets overall hard to predict:  0.12485945117235554


In [3]:
from matplotlib.colors import LogNorm
                
fig, (ax1, ax2) = plt.subplots(figsize=(5, 4), ncols=2, gridspec_kw={'width_ratios': [1, 0.1]})

H, xedges, yedges = np.histogram2d(distances_filtered, noise_filtered, bins=[np.linspace(0, 100, 100), np.linspace(0, 1, 100)])
hex_ax = ax1.pcolor(xedges, yedges, H.T, norm=LogNorm(), cmap='jet', rasterized=True, zorder=10)

ax1.set_facecolor('black')
ax1.set_yticks(np.arange(0, 1.01, 0.05))
ax1.set_xticks(np.arange(0, 101, 5))
y_labels = [str(i/100) if i % 20 == 0 else "" for i in range(0, 101, 5)]
x_labels = [str(i) if i % 20 == 0 else "" for i in range(0, 101, 5)]
ax1.set_yticklabels(y_labels)
ax1.set_xticklabels(x_labels)
ax1.set_ylim(0, 1)
ax1.set_xlim(0, 100)
ax1.set_ylabel('Wait Time Diversity (%)')
ax1.set_xlabel('Cycle Discrepancy (s)')

# Add colorbar
cbar = fig.colorbar(hex_ax, cax=ax2)
cbar.set_label('\# Hourly Buckets')

fig_name = 'figures/predictability-heatmap'
plt.savefig(f'{fig_name}.pdf', bbox_inches='tight')
plt.savefig(f'{fig_name}.png', bbox_inches='tight', dpi=300)
plt.savefig(f'{fig_name}.pgf', bbox_inches='tight', dpi=300)

In [4]:
import tempfile
import zipfile
import os

with open('../batch_processing/processed_things_2024_01_22.zip', 'rb') as f:
    # Unzip in tempdir and load json
    with tempfile.TemporaryDirectory() as temp_dir:
        with zipfile.ZipFile(f, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        with open(os.path.join(temp_dir, "processed_things_2024_01_22.json")) as f:
            processed_things = json.load(f)

# load from json
with open('data/things.json') as json_file:
    things = json.load(json_file)

# Convert things into a dict
things_dict = {}
for thing in things:
    things_dict[thing["name"]] = thing

In [5]:
from lib.mapbox import fetch_city_map
from pyproj import Transformer

WGS_TO_MERCATOR = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
MERCATOR_TO_WGS = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

# Center region of Hamburg
center_lat_ax1, center_lng_ax1 = 53.5515, 9.9769
min_lat_ax1, min_lng_ax1 = center_lat_ax1 - 0.05, center_lng_ax1 - 0.05
max_lat_ax1, max_lng_ax1 = center_lat_ax1 + 0.05, center_lng_ax1 + 0.05
center_lat_ax1_zoom, center_lng_ax1_zoom = 53.556166, 9.977955
min_lat_ax1_zoom, min_lng_ax1_zoom = center_lat_ax1_zoom - 0.002, center_lng_ax1_zoom - 0.002
max_lat_ax1_zoom, max_lng_ax1_zoom = center_lat_ax1_zoom + 0.002, center_lng_ax1_zoom + 0.002

min_wm_x_ax1, min_wm_y_ax1 = WGS_TO_MERCATOR.transform(min_lng_ax1, min_lat_ax1)
max_wm_x_ax1, max_wm_y_ax1 = WGS_TO_MERCATOR.transform(max_lng_ax1, max_lat_ax1)
min_wm_x_ax1_zoom, min_wm_y_ax1_zoom = WGS_TO_MERCATOR.transform(min_lng_ax1_zoom, min_lat_ax1_zoom)
max_wm_x_ax1_zoom, max_wm_y_ax1_zoom = WGS_TO_MERCATOR.transform(max_lng_ax1_zoom, max_lat_ax1_zoom)

# Make the bounding box square
wm_x_diff_ax1 = max_wm_x_ax1 - min_wm_x_ax1
wm_y_diff_ax1 = max_wm_y_ax1 - min_wm_y_ax1
if wm_x_diff_ax1 > wm_y_diff_ax1:
    min_wm_y_ax1 -= (wm_x_diff_ax1 - wm_y_diff_ax1) / 2
    max_wm_y_ax1 += (wm_x_diff_ax1 - wm_y_diff_ax1) / 2
else:
    min_wm_x_ax1 -= (wm_y_diff_ax1 - wm_x_diff_ax1) / 2
    max_wm_x_ax1 += (wm_y_diff_ax1 - wm_x_diff_ax1) / 2
wm_x_diff_ax1_zoom = max_wm_x_ax1_zoom - min_wm_x_ax1_zoom
wm_y_diff_ax1_zoom = max_wm_y_ax1_zoom - min_wm_y_ax1_zoom
if wm_x_diff_ax1_zoom > wm_y_diff_ax1_zoom:
    min_wm_y_ax1_zoom -= (wm_x_diff_ax1_zoom - wm_y_diff_ax1_zoom) / 2
    max_wm_y_ax1_zoom += (wm_x_diff_ax1_zoom - wm_y_diff_ax1_zoom) / 2
else:
    min_wm_x_ax1_zoom -= (wm_y_diff_ax1_zoom - wm_x_diff_ax1_zoom) / 2
    max_wm_x_ax1_zoom += (wm_y_diff_ax1_zoom - wm_x_diff_ax1_zoom) / 2

# Fetch map
img_min_lng_ax1, img_min_lat_ax1 = MERCATOR_TO_WGS.transform(min_wm_x_ax1, min_wm_y_ax1)
img_max_lng_ax1, img_max_lat_ax1 = MERCATOR_TO_WGS.transform(max_wm_x_ax1, max_wm_y_ax1)
map_img_ax1 = fetch_city_map(img_min_lat_ax1, img_min_lng_ax1, img_max_lat_ax1, img_max_lng_ax1, style='light-v10')
img_min_lng_ax1_zoom, img_min_lat_ax1_zoom = MERCATOR_TO_WGS.transform(min_wm_x_ax1_zoom, min_wm_y_ax1_zoom)
img_max_lng_ax1_zoom, img_max_lat_ax1_zoom = MERCATOR_TO_WGS.transform(max_wm_x_ax1_zoom, max_wm_y_ax1_zoom)
map_img_ax1_zoom = fetch_city_map(img_min_lat_ax1_zoom, img_min_lng_ax1_zoom, img_max_lat_ax1_zoom, img_max_lng_ax1_zoom, style='light-v10')

fig, (ax1) = plt.subplots(figsize=(8, 6), ncols=1)
ax1.imshow(map_img_ax1, extent=[min_wm_x_ax1, max_wm_x_ax1, min_wm_y_ax1, max_wm_y_ax1], aspect='1', zorder=0, interpolation='bilinear')

# Add a subplot on the lower left that zooms in to the area of interest
ax1_focus = fig.add_axes([0.1458, 0.11, 0.33, 0.31])
ax1_focus.imshow(map_img_ax1_zoom, extent=[min_wm_x_ax1_zoom, max_wm_x_ax1_zoom, min_wm_y_ax1_zoom, max_wm_y_ax1_zoom], aspect='1', zorder=0, interpolation='bilinear')

ax1.plot([min_wm_x_ax1_zoom, min_wm_x_ax1_zoom], [min_wm_y_ax1_zoom, max_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')
ax1.plot([max_wm_x_ax1_zoom, max_wm_x_ax1_zoom], [min_wm_y_ax1_zoom, max_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')
ax1.plot([min_wm_x_ax1_zoom, max_wm_x_ax1_zoom], [min_wm_y_ax1_zoom, min_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')
ax1.plot([min_wm_x_ax1_zoom, max_wm_x_ax1_zoom], [max_wm_y_ax1_zoom, max_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')

focus_axes_top_left_y =  0.4 * (max_wm_y_ax1 - min_wm_y_ax1) + min_wm_y_ax1
focus_axes_top_left_x = min_wm_x_ax1
focus_axes_top_right_y =  0.4 * (max_wm_y_ax1 - min_wm_y_ax1) + min_wm_y_ax1
focus_axes_top_right_x = 0.4 * (max_wm_x_ax1 - min_wm_x_ax1) + min_wm_x_ax1
focus_axes_bottom_left_y = min_wm_y_ax1
focus_axes_bottom_left_x = min_wm_x_ax1
focus_axes_bottom_right_y = min_wm_y_ax1
focus_axes_bottom_right_x = 0.4 * (max_wm_x_ax1 - min_wm_x_ax1) + min_wm_x_ax1

ax1.plot([focus_axes_top_left_x, min_wm_x_ax1_zoom], [focus_axes_top_left_y, max_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')
ax1.plot([focus_axes_top_right_x, max_wm_x_ax1_zoom], [focus_axes_top_right_y, max_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')
ax1.plot([focus_axes_bottom_left_x, min_wm_x_ax1_zoom], [focus_axes_bottom_left_y, min_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')
ax1.plot([focus_axes_bottom_right_x, max_wm_x_ax1_zoom], [focus_axes_bottom_right_y, min_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')

ax1.set_xlim(min_wm_x_ax1, max_wm_x_ax1)
ax1.set_ylim(min_wm_y_ax1, max_wm_y_ax1)
ax1.set_xticks([])
ax1.set_yticks([])

ax1_focus.set_xlim(min_wm_x_ax1_zoom, max_wm_x_ax1_zoom)
ax1_focus.set_ylim(min_wm_y_ax1_zoom, max_wm_y_ax1_zoom)
ax1_focus.set_xticks([])
ax1_focus.set_yticks([])

for i, (thing_name, thing) in enumerate(processed_things.items()):
    if not "primary" in thing_name:
        continue
    if thing["TotalCyclesCount"] == 0:
        continue
    if thing["TotalRemovedCycleCount"] / thing["TotalCyclesCount"] > 0.1:
        continue
    thing_meta = things_dict[thing_name.replace("_primary", "")]

    # Calculate median distance
    distances = []
    for day_idx in range(7):
        for hour_idx in range(24):
            if thing["Metrics"][day_idx][hour_idx] != -1.0:
                distances.append(thing["Metrics"][day_idx][hour_idx])
    median_distance = np.median(distances)

    for location in thing_meta["Locations"]:
        geometry = location['location']['geometry']
        for linestring in geometry['coordinates']:
            cmap = plt.get_cmap('jet')
            # Log space for distance
            color = cmap(max(0, min(1, median_distance / 50)))
            ax1.plot(*WGS_TO_MERCATOR.transform(*np.array(linestring).T), color=color, linewidth=1, zorder=1, rasterized=True)
            ax1_focus.plot(*WGS_TO_MERCATOR.transform(*np.array(linestring).T), color=color, linewidth=1, zorder=1, rasterized=True)

plt.subplots_adjust(wspace=0.03, hspace=0.1)

# Add colorbar
cmap = plt.get_cmap('jet')
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=50))
sm.set_array([])
cb = fig.colorbar(sm, ax=ax1, orientation='vertical', pad=0.015, shrink=1)
# fontsize
cb.ax.tick_params(labelsize=12)
cb.set_label('Cycle Discrepancy (s)', fontsize=12)
cb.ax.xaxis.set_ticks_position('top')

fig_name = f'figures/predictability-map'
plt.savefig(f'{fig_name}.pdf', bbox_inches='tight', dpi=300)
plt.savefig(f'{fig_name}.png', bbox_inches='tight', dpi=300)
plt.savefig(f'{fig_name}.pgf', bbox_inches='tight', dpi=300)

/Users/philippmatthes/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
from lib.mapbox import fetch_city_map
from pyproj import Transformer

WGS_TO_MERCATOR = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
MERCATOR_TO_WGS = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

# Center region of Hamburg
center_lat_ax1, center_lng_ax1 = 53.5515, 9.9769
min_lat_ax1, min_lng_ax1 = center_lat_ax1 - 0.05, center_lng_ax1 - 0.05
max_lat_ax1, max_lng_ax1 = center_lat_ax1 + 0.05, center_lng_ax1 + 0.05
center_lat_ax1_zoom, center_lng_ax1_zoom = 53.556166, 9.977955
min_lat_ax1_zoom, min_lng_ax1_zoom = center_lat_ax1_zoom - 0.002, center_lng_ax1_zoom - 0.002
max_lat_ax1_zoom, max_lng_ax1_zoom = center_lat_ax1_zoom + 0.002, center_lng_ax1_zoom + 0.002

min_wm_x_ax1, min_wm_y_ax1 = WGS_TO_MERCATOR.transform(min_lng_ax1, min_lat_ax1)
max_wm_x_ax1, max_wm_y_ax1 = WGS_TO_MERCATOR.transform(max_lng_ax1, max_lat_ax1)
min_wm_x_ax1_zoom, min_wm_y_ax1_zoom = WGS_TO_MERCATOR.transform(min_lng_ax1_zoom, min_lat_ax1_zoom)
max_wm_x_ax1_zoom, max_wm_y_ax1_zoom = WGS_TO_MERCATOR.transform(max_lng_ax1_zoom, max_lat_ax1_zoom)

# Make the bounding box square
wm_x_diff_ax1 = max_wm_x_ax1 - min_wm_x_ax1
wm_y_diff_ax1 = max_wm_y_ax1 - min_wm_y_ax1
if wm_x_diff_ax1 > wm_y_diff_ax1:
    min_wm_y_ax1 -= (wm_x_diff_ax1 - wm_y_diff_ax1) / 2
    max_wm_y_ax1 += (wm_x_diff_ax1 - wm_y_diff_ax1) / 2
else:
    min_wm_x_ax1 -= (wm_y_diff_ax1 - wm_x_diff_ax1) / 2
    max_wm_x_ax1 += (wm_y_diff_ax1 - wm_x_diff_ax1) / 2
wm_x_diff_ax1_zoom = max_wm_x_ax1_zoom - min_wm_x_ax1_zoom
wm_y_diff_ax1_zoom = max_wm_y_ax1_zoom - min_wm_y_ax1_zoom
if wm_x_diff_ax1_zoom > wm_y_diff_ax1_zoom:
    min_wm_y_ax1_zoom -= (wm_x_diff_ax1_zoom - wm_y_diff_ax1_zoom) / 2
    max_wm_y_ax1_zoom += (wm_x_diff_ax1_zoom - wm_y_diff_ax1_zoom) / 2
else:
    min_wm_x_ax1_zoom -= (wm_y_diff_ax1_zoom - wm_x_diff_ax1_zoom) / 2
    max_wm_x_ax1_zoom += (wm_y_diff_ax1_zoom - wm_x_diff_ax1_zoom) / 2

# Fetch map
img_min_lng_ax1, img_min_lat_ax1 = MERCATOR_TO_WGS.transform(min_wm_x_ax1, min_wm_y_ax1)
img_max_lng_ax1, img_max_lat_ax1 = MERCATOR_TO_WGS.transform(max_wm_x_ax1, max_wm_y_ax1)
map_img_ax1 = fetch_city_map(img_min_lat_ax1, img_min_lng_ax1, img_max_lat_ax1, img_max_lng_ax1, style='light-v10')
img_min_lng_ax1_zoom, img_min_lat_ax1_zoom = MERCATOR_TO_WGS.transform(min_wm_x_ax1_zoom, min_wm_y_ax1_zoom)
img_max_lng_ax1_zoom, img_max_lat_ax1_zoom = MERCATOR_TO_WGS.transform(max_wm_x_ax1_zoom, max_wm_y_ax1_zoom)
map_img_ax1_zoom = fetch_city_map(img_min_lat_ax1_zoom, img_min_lng_ax1_zoom, img_max_lat_ax1_zoom, img_max_lng_ax1_zoom, style='light-v10')

fig, (ax1) = plt.subplots(figsize=(8, 6), ncols=1)
ax1.imshow(map_img_ax1, extent=[min_wm_x_ax1, max_wm_x_ax1, min_wm_y_ax1, max_wm_y_ax1], aspect='1', zorder=0, interpolation='bilinear')

# Add a subplot on the lower left that zooms in to the area of interest
ax1_focus = fig.add_axes([0.1458, 0.11, 0.33, 0.31])
ax1_focus.imshow(map_img_ax1_zoom, extent=[min_wm_x_ax1_zoom, max_wm_x_ax1_zoom, min_wm_y_ax1_zoom, max_wm_y_ax1_zoom], aspect='1', zorder=0, interpolation='bilinear')

ax1.plot([min_wm_x_ax1_zoom, min_wm_x_ax1_zoom], [min_wm_y_ax1_zoom, max_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')
ax1.plot([max_wm_x_ax1_zoom, max_wm_x_ax1_zoom], [min_wm_y_ax1_zoom, max_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')
ax1.plot([min_wm_x_ax1_zoom, max_wm_x_ax1_zoom], [min_wm_y_ax1_zoom, min_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')
ax1.plot([min_wm_x_ax1_zoom, max_wm_x_ax1_zoom], [max_wm_y_ax1_zoom, max_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')

focus_axes_top_left_y =  0.4 * (max_wm_y_ax1 - min_wm_y_ax1) + min_wm_y_ax1
focus_axes_top_left_x = min_wm_x_ax1
focus_axes_top_right_y =  0.4 * (max_wm_y_ax1 - min_wm_y_ax1) + min_wm_y_ax1
focus_axes_top_right_x = 0.4 * (max_wm_x_ax1 - min_wm_x_ax1) + min_wm_x_ax1
focus_axes_bottom_left_y = min_wm_y_ax1
focus_axes_bottom_left_x = min_wm_x_ax1
focus_axes_bottom_right_y = min_wm_y_ax1
focus_axes_bottom_right_x = 0.4 * (max_wm_x_ax1 - min_wm_x_ax1) + min_wm_x_ax1

ax1.plot([focus_axes_top_left_x, min_wm_x_ax1_zoom], [focus_axes_top_left_y, max_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')
ax1.plot([focus_axes_top_right_x, max_wm_x_ax1_zoom], [focus_axes_top_right_y, max_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')
ax1.plot([focus_axes_bottom_left_x, min_wm_x_ax1_zoom], [focus_axes_bottom_left_y, min_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')
ax1.plot([focus_axes_bottom_right_x, max_wm_x_ax1_zoom], [focus_axes_bottom_right_y, min_wm_y_ax1_zoom], color='black', linewidth=1, zorder=1, linestyle='-')

ax1.set_xlim(min_wm_x_ax1, max_wm_x_ax1)
ax1.set_ylim(min_wm_y_ax1, max_wm_y_ax1)
ax1.set_xticks([])
ax1.set_yticks([])

ax1_focus.set_xlim(min_wm_x_ax1_zoom, max_wm_x_ax1_zoom)
ax1_focus.set_ylim(min_wm_y_ax1_zoom, max_wm_y_ax1_zoom)
ax1_focus.set_xticks([])
ax1_focus.set_yticks([])

for i, (thing_name, thing) in enumerate(processed_things.items()):
    if not "primary" in thing_name:
        continue
    if thing["TotalCyclesCount"] == 0:
        continue
    if thing["TotalRemovedCycleCount"] / thing["TotalCyclesCount"] > 0.1:
        continue
    thing_meta = things_dict[thing_name.replace("_primary", "")]

    # Calculate median wait time diversity
    diversities = []
    for day_idx in range(7):
        for hour_idx in range(24):
            if thing["ShiftsFuzzyness"][day_idx][hour_idx] != -1.0:
                diversities.append(thing["ShiftsFuzzyness"][day_idx][hour_idx])
    median_diversity = np.median(diversities)

    for location in thing_meta["Locations"]:
        geometry = location['location']['geometry']
        for linestring in geometry['coordinates']:
            cmap = plt.get_cmap('jet')
            color = cmap(max(0, min(1, median_diversity)))
            ax1.plot(*WGS_TO_MERCATOR.transform(*np.array(linestring).T), color=color, linewidth=1, zorder=1, rasterized=True)
            ax1_focus.plot(*WGS_TO_MERCATOR.transform(*np.array(linestring).T), color=color, linewidth=1, zorder=1, rasterized=True)

plt.subplots_adjust(wspace=0.03, hspace=0.1)

# Add colorbar
cmap = plt.get_cmap('jet')
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=1))
sm.set_array([])
cb = fig.colorbar(sm, ax=ax1, orientation='vertical', pad=0.015, shrink=1)
# fontsize
cb.ax.tick_params(labelsize=12)
cb.set_label('Wait Time Diversity (\%)', fontsize=12)
cb.ax.xaxis.set_ticks_position('top')
# Set labels to 0%, 20%, ..., 100%
cb.ax.set_yticklabels([f'{i*20}' for i in range(6)])

fig_name = f'figures/predictability-map-diversity'
plt.savefig(f'{fig_name}.pdf', bbox_inches='tight', dpi=300)
plt.savefig(f'{fig_name}.png', bbox_inches='tight', dpi=300)
plt.savefig(f'{fig_name}.pgf', bbox_inches='tight', dpi=300)

/Users/philippmatthes/Library/Python/3.9/lib/python/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/philippmatthes/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/dz/rdg3g32d6flc0twpctgtzjz80000gn/T/ipykernel_10548/4229210650.py:119: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  cb.ax.set_yticklabels([f'{i*20}' for i in range(6)])
